In [149]:
import json
import pandas as pd

In [150]:
filename = 'producto_PEEsika_20201012.json'
f = open(filename,) 
data = json.load(f) 

In [151]:
def format_row(data_prod, flag_ecommerce=True):
    if ('price' in data_prod):        
        return_data = {}
        return_data['codpais']= data_prod['price']['currencyIso'][0:2] 
        return_data['codsap'] = data_prod['code']
        return_data['categories'] = [cat['code'] for cat in data_prod['categories']]
        if ('images' in data_prod):
            return_data['images_url'] = max([image['url']  if image['format'] == 'product' else '' for image in data_prod['images']])
        else: 
            return_data['images_url'] = ''
        return_data['isVariableSet'] = data_prod['isVariableSet']
        if ('precioTachado' in data_prod): 
            return_data['price'] = data_prod['precioTachado']['value']
        else: 
            return_data['price'] = ''
        return_data['sale_price'] = data_prod['price']['value']
        return_data['stockLevelStatus'] = data_prod['stock']['stockLevelStatus']['value']
        return_data['link_url'] = data_prod['url']
        return_data['flag_ecommerce'] = flag_ecommerce
        return_data['name_prod'] = data_prod.get('name')
        #print(return_data)
        return return_data

In [152]:
return_data = pd.DataFrame({object:None,object:None,object:None,object:None,object:None,object:None,object:None,object:None,object:None,object:None}, 
                        columns = ['codpais','codsap','categories','images_url', 'isVariableSet','price','sale_price','stockLevelStatus','link_url'])
for data_prod in data['products']:
    if ('price' in data_prod):
        sap_prd = pd.json_normalize(format_row(data_prod))
    return_data = return_data.append(sap_prd) 


In [153]:
len(return_data)

1216

In [154]:
return_data.to_csv(filename.replace('json','csv'), index=False)

In [155]:
def load_json_fiile(filename):
    f = open(filename,) 
    data = json.load(f) 
    print(filename)
    return data

In [156]:
def process_row(data):
    return_data = pd.DataFrame({object:None,object:None,object:None,object:None,object:None,object:None,object:None,object:None,object:None,object:None}, 
                        columns = ['codpais','codsap','categories','images_url', 'isVariableSet','price','sale_price','stockLevelStatus','link_url','flag_ecommerce'])
    for data_prod in data['products']:
        if ('price' in data_prod):
            sap_prd = pd.json_normalize(format_row(data_prod))
            return_data = return_data.append(sap_prd)
        if ( str.lower(data_prod['code'][-2:]) in country):
            data_prod['price'] = {'currencyIso':str.upper(data_prod['code'][-2:]), 'value':0.0}
            #data_prod['precioTachado']['value'] = 0.0
            sap_prd = pd.json_normalize(format_row(data_prod, False))
            return_data = return_data.append(sap_prd)
            
        
    return return_data

In [157]:
final_data = pd.DataFrame({object:None,object:None,object:None,object:None,object:None,object:None,object:None,object:None,object:None,object:None}, 
                        columns = ['codpais','codsap','categories','images_url', 'isVariableSet','price','sale_price','stockLevelStatus','link_url'])


In [158]:
files_name = ['producto_CLCyzone_20201008.json','producto_COCyzone_20201008.json','producto_MXCyzone_20201008.json',
              'producto_PALbel_20201008.json','producto_PELbel_20201008.json', 'producto_CLEsika_20201008.json', 
              'producto_COEsika_20201008.json', 'producto_MXEsika_20201008.json', 'producto_PECyzone_20201008.json',
              'producto_CLLbel_20201008.json',  'producto_COLbel_20201008.json',  'producto_MXLbel_20201008.json',
              'producto_PEEsika_20201008.json']

country = ['pe','co','cl','mx']

for filename in files_name:
    mdata = load_json_fiile(filename)
    return_data= process_row(mdata)
    final_data = final_data.append(return_data)
    

producto_CLCyzone_20201008.json
producto_COCyzone_20201008.json
producto_MXCyzone_20201008.json
producto_PALbel_20201008.json
producto_PELbel_20201008.json
producto_CLEsika_20201008.json
producto_COEsika_20201008.json
producto_MXEsika_20201008.json
producto_PECyzone_20201008.json
producto_CLLbel_20201008.json
producto_COLbel_20201008.json
producto_MXLbel_20201008.json
producto_PEEsika_20201008.json


In [159]:
def format_subcategories(jsondata, lista_return, category_parent):    
    if 'name' in jsondata:
        node_id = jsondata['id']
        node_name = jsondata['name']        
    
        if category_parent == '':
            #print('{"'+node_id+'":{"properties":{"name":"'+node_name+'"}}}')
            lista_return.append(json.loads('{"'+node_id+'":{"properties":{"name":"'+node_name+'"}}}'))
        else:
            #print('{"'+node_id+'":{"properties":{"name":"'+node_name+'"},"associations":{"category":{"parent":["'+category_parent+'"]}}}}')
            lista_return.append(json.loads('{"'+node_id+'":{"properties":{"name":"'+node_name+'"},"associations":{"category":{"parent":["'+category_parent+'"]}}}}'))
              
    if 'subcategories' in jsondata:
          for child in jsondata['subcategories']:
              format_subcategories(child,lista_return,node_id)           

In [160]:
final_data[final_data['codsap']=='210090365']

,codpais,codsap,categories,images_url,isVariableSet,price,sale_price,stockLevelStatus,link_url,flag_ecommerce,name_prod
0,CO,210090365,"[esika-07050104, esika-50]",https://cdn1-prd.beautymovers.com/sys-master/s...,False,60000,56999.9,inStock,https://esika.tiendabelcorp.com/co/estuche-are...,True,Estuche de Aretes x2 Chiara
0,MX,210090365,[esika-07050102],https://cdn1-prd.beautymovers.com/sys-master/s...,False,500,474.9,inStock,https://esika.tiendabelcorp.com/mx//p/210090365,True,None
0,PE,210090365,[esika-07050102],https://cdn1-prd.beautymovers.com/sys-master/s...,False,80,75.9,outOfStock,https://esika.tiendabelcorp.com/pe//p/210090365,True,None


In [161]:
final_data[final_data['codsap']=='200100838']

,codpais,codsap,categories,images_url,isVariableSet,price,sale_price,stockLevelStatus,link_url,flag_ecommerce,name_prod
0,CL,200100838,[esika-020203],https://cdn1-prd.beautymovers.com/sys-master/s...,False,36600,29190.0,inStock,https://esika.tiendabelcorp.com/cl/magnat/p/20...,True,Magnat
0,CO,200100838,"[esika-020203, esika-31]",https://cdn1-prd.beautymovers.com/sys-master/s...,False,142000,112900.0,inStock,https://esika.tiendabelcorp.com/co/magnat/p/20...,True,Magnat
0,MX,200100838,"[esika-020203, esika-10]",https://cdn1-prd.beautymovers.com/sys-master/s...,False,710,567.9,inStock,https://esika.tiendabelcorp.com/mx//p/200100838,True,None
0,PE,200100838,[esika-020203],https://cdn1-prd.beautymovers.com/sys-master/s...,False,164,130.9,inStock,https://esika.tiendabelcorp.com/pe//p/200100838,True,None


In [162]:
data_cyzone = load_json_fiile('categoria_peCyzone.json')
data_esika = load_json_fiile('categoria_PEEsika_20201008.json')
data_lbel = load_json_fiile('categoria_peLbel.json')


categoria_peCyzone.json
categoria_PEEsika_20201008.json
categoria_peLbel.json


In [163]:
categorias = []
format_subcategories(data_cyzone['catalogVersions'][0]['categories'][0], categorias, '')
format_subcategories(data_esika['catalogVersions'][0]['categories'][0], categorias, '')
format_subcategories(data_lbel['catalogVersions'][0]['categories'][0], categorias, '')


In [164]:
def look_categorias(main_categorias, categoria, lista):
    for row in main_categorias:
        data = row.get(categoria)
        if data != None:
            lista.append(categoria)
            if data.get('associations'):
                look_categorias(main_categorias, data['associations']['category']['parent'][0],lista)

In [165]:
prod_catg_list = []
look_categorias(categorias, 'esika-030401',prod_catg_list)
prod_catg_list

['esika-030401', 'esika-0304', 'esika-03', 'esika']

In [166]:
def filter_categories(list_categories):
    prod_catg_list =[]
    for categ in list_categories:
        look_categorias(categorias, categ,prod_catg_list)
    return set(prod_catg_list)

In [167]:
filter_categories(['esika-0304', 'esika-0305', 'esika-0308'])

{'esika', 'esika-03', 'esika-0304', 'esika-0305'}

In [169]:
final_data.columns.get_loc('categories')

2

In [170]:
idx_categ = final_data.columns.get_loc('categories')
for row in range(0, len(final_data)):
    final_data.iat[row, idx_categ]= filter_categories(final_data.iat[row, idx_categ]) 

In [171]:
final_data[final_data['codsap']=='200098640']

,codpais,codsap,categories,images_url,isVariableSet,price,sale_price,stockLevelStatus,link_url,flag_ecommerce,name_prod
0,CL,200098640,"{esika-0305, esika-03, esika, esika-0309}",https://cdn1-prd.beautymovers.com/sys-master/s...,False,21000,16790.0,inStock,https://esika.tiendabelcorp.com/cl/contorno-oj...,True,Contorno de Ojos Oxygen Supreme
0,CO,200098640,"{esika-0305, esika-03, esika, esika-0309}",https://cdn1-prd.beautymovers.com/sys-master/s...,False,69000,54900.0,outOfStock,https://esika.tiendabelcorp.com/co/contorno-oj...,True,Contorno de Ojos Oxygen Supreme
0,PE,200098640,"{esika-0305, esika-03, esika, esika-0309}",https://cdn1-prd.beautymovers.com/sys-master/s...,False,93,73.9,inStock,https://esika.tiendabelcorp.com/pe//p/200098640,True,None


In [172]:
final_data[final_data['codsap']=='200100838']

,codpais,codsap,categories,images_url,isVariableSet,price,sale_price,stockLevelStatus,link_url,flag_ecommerce,name_prod
0,CL,200100838,"{esika, esika-0202, esika-02, esika-020203}",https://cdn1-prd.beautymovers.com/sys-master/s...,False,36600,29190.0,inStock,https://esika.tiendabelcorp.com/cl/magnat/p/20...,True,Magnat
0,CO,200100838,"{esika, esika-0202, esika-02, esika-020203}",https://cdn1-prd.beautymovers.com/sys-master/s...,False,142000,112900.0,inStock,https://esika.tiendabelcorp.com/co/magnat/p/20...,True,Magnat
0,MX,200100838,"{esika-0202, esika-02, esika, esika-10, esika-...",https://cdn1-prd.beautymovers.com/sys-master/s...,False,710,567.9,inStock,https://esika.tiendabelcorp.com/mx//p/200100838,True,None
0,PE,200100838,"{esika, esika-0202, esika-02, esika-020203}",https://cdn1-prd.beautymovers.com/sys-master/s...,False,164,130.9,inStock,https://esika.tiendabelcorp.com/pe//p/200100838,True,None


In [173]:
final_data.iat[0,2 ]

{'cyzone', 'cyzone-01', 'cyzone-0101', 'cyzone-0107'}

In [136]:
#final_data.to_csv('product_hybris_20201012.csv', index=False)

In [137]:
final_data['codsap']

0    210094882
0    210095586
0    210094932
0    210094927
0    210095984
       ...    
0    200100673
0    200060841
0    200099529
0    200098267
0    200086568
Name: codsap, Length: 9620, dtype: object

In [176]:
final_data.columns.get_loc("flagSetHybris")

11

In [175]:
final_data['flagSetHybris']=False
idx_seth= final_data.columns.get_loc("flagSetHybris")
for row in range(0, len(final_data)):
    if '-' in final_data.iat[row,1]:
        print("insert into #tac_ecom values( '",final_data.iat[row,0],"','",str.split(final_data.iat[row,1],'-')[0],"','",str.split(final_data.iat[row,1],'-')[1],"');")
        final_data.iat[row,idx_seth] = True

insert into #tac_ecom values( ' CL ',' 29030 ',' 202013 ');
insert into #tac_ecom values( ' CL ',' 30156 ',' 202010 ');
insert into #tac_ecom values( ' CL ',' 30152 ',' 202010 ');
insert into #tac_ecom values( ' CL ',' 29010 ',' 202015 ');
insert into #tac_ecom values( ' CL ',' 30262 ',' 202010 ');
insert into #tac_ecom values( ' CO ',' 29259 ',' 202013 ');
insert into #tac_ecom values( ' CO ',' 30009 ',' 202013 ');
insert into #tac_ecom values( ' CO ',' 70319 ',' 202010 ');
insert into #tac_ecom values( ' CO ',' 70321 ',' 202010 ');
insert into #tac_ecom values( ' CO ',' 29190 ',' 202015 ');
insert into #tac_ecom values( ' CO ',' 70326 ',' 202010 ');
insert into #tac_ecom values( ' CO ',' 29258 ',' 202013 ');
insert into #tac_ecom values( ' MX ',' 29029 ',' 202013 ');
insert into #tac_ecom values( ' MX ',' 29027 ',' 202013 ');
insert into #tac_ecom values( ' MX ',' 29108 ',' 202015 ');
insert into #tac_ecom values( ' MX ',' 30478 ',' 202010 ');
insert into #tac_ecom values( ' MX ',' 3

In [177]:
final_data[final_data['flagSetHybris']]

,codpais,codsap,categories,images_url,isVariableSet,price,sale_price,stockLevelStatus,link_url,flag_ecommerce,name_prod,flagSetHybris
0,CL,29030-202013,"{cyzone-40, cyzone}",https://cdn1-prd.beautymovers.com/sys-master/s...,False,23500,18790.0,inStock,https://cyzone.tiendabelcorp.com/cl/set-comple...,True,Set completo skin first,True
0,CL,30156-202010,"{cyzone-40, cyzone}",https://cdn1-prd.beautymovers.com/sys-master/s...,False,25200,20090.0,outOfStock,https://cyzone.tiendabelcorp.com/cl/set-sweet-...,True,Set sweet chic,True
0,CL,30152-202010,"{cyzone-40, cyzone}",https://cdn1-prd.beautymovers.com/sys-master/s...,False,25200,20090.0,outOfStock,https://cyzone.tiendabelcorp.com/cl/set-sweet-...,True,Set sweet black,True
0,CL,29010-202015,"{cyzone-40, cyzone}",https://cdn1-prd.beautymovers.com/sys-master/s...,False,28600,22790.0,inStock,https://cyzone.tiendabelcorp.com/cl/set-spray-...,True,Set spray fijador + paleta studio look,True
0,CL,30262-202010,"{cyzone-40, cyzone}",https://cdn1-prd.beautymovers.com/sys-master/s...,False,31400,24990.0,inStock,https://cyzone.tiendabelcorp.com/cl/set-studio...,True,Set studio look,True
...,...,...,...,...,...,...,...,...,...,...,...,...
0,PE,29090-202009,"{esika-40, esika-08, esika}",https://cdn1-prd.beautymovers.com/sys-master/s...,False,280,223.9,inStock,https://esika.tiendabelcorp.com/pe/set-magnat-...,True,Set Magnat Opción 03,True
0,PE,29067-202009,"{esika-40, esika-08, esika}",https://cdn1-prd.beautymovers.com/sys-master/s...,False,150,119.9,inStock,https://esika.tiendabelcorp.com/pe/set-motion/...,True,Set Motion,True
0,PE,05695-202009,"{esika-40, esika-08, esika}",https://cdn1-prd.beautymovers.com/sys-master/s...,False,151,119.9,inStock,https://esika.tiendabelcorp.com/pe/set-you-tra...,True,Set You Travel,True
0,PE,29233-202010,"{esika-40, esika}",https://cdn1-prd.beautymovers.com/sys-master/s...,False,211,159.9,inStock,https://esika.tiendabelcorp.com/pe//p/29233-20...,True,None,True


In [178]:
df_setHybris_codsap = pd.read_csv('setHybris_codsap.csv')

In [179]:
df_setHybris_codsap.head()

,codpais,codventa,aniocampana,codsap,indpadre,rr_cant
0,CO,70319,202010,200094149,1,1
1,CL,29010,202015,200096224,1,1
2,CL,29222,202009,200088132,1,1
3,CL,29152,202010,200095931,1,1
4,CL,29220,202009,200086396,1,1


In [180]:
df_setHybris_codsap.describe()

,codventa,aniocampana,codsap,indpadre,rr_cant
count,83.000000,83.000000,8.300000e+01,83.0,83.000000
mean,30721.759036,202010.084337,2.000912e+08,1.0,1.012048
std,9640.784901,1.683527,1.061293e+04,0.0,0.109764
min,5695.000000,202009.000000,2.000299e+08,1.0,1.000000
25%,29102.000000,202009.000000,2.000873e+08,1.0,1.000000
50%,29241.000000,202009.000000,2.000945e+08,1.0,1.000000
75%,29390.500000,202010.000000,2.000979e+08,1.0,1.000000
max,70326.000000,202015.000000,2.001009e+08,1.0,2.000000


In [181]:
def searchset_codsap(df_setHybris, codpais, codproducto):
    codventa= int(codproducto.split('-')[0])
    aniocampana= int(codproducto.split('-')[1])
    print(codventa,aniocampana)
    datasap = df_setHybris_codsap[(df_setHybris_codsap['codpais']==codpais) & 
                               (df_setHybris_codsap['aniocampana']==aniocampana) & 
                               (df_setHybris_codsap['codventa']==codventa)]
    if len(datasap)>0:
        return datasap.iat[0, 3]
    else:
        return ''

In [119]:
searchset_codsap(df_setHybris_codsap,'PE', '29133-202013')

29133 202013


200097894

In [ ]:
final_data.columns.get_loc("codsap_parent")

In [182]:
final_data['codsap_parent']=''
idx_codsapParent= final_data.columns.get_loc("codsap_parent")

for row in range(0, len(final_data)):
    if '-' in final_data.iat[row,1]:
        print(final_data.iat[row,0],",",str.split(final_data.iat[row,1],'-')[0],"','",str.split(final_data.iat[row,1],'-')[1],"');")
        codpais = final_data.iat[row,0]
        codproducto = final_data.iat[row,1]
        final_data.iat[row, idx_codsapParent] = searchset_codsap(df_setHybris_codsap,codpais, codproducto)

CL , 29030 ',' 202013 ');
29030 202013
CL , 30156 ',' 202010 ');
30156 202010
CL , 30152 ',' 202010 ');
30152 202010
CL , 29010 ',' 202015 ');
29010 202015
CL , 30262 ',' 202010 ');
30262 202010
CO , 29259 ',' 202013 ');
29259 202013
CO , 30009 ',' 202013 ');
30009 202013
CO , 70319 ',' 202010 ');
70319 202010
CO , 70321 ',' 202010 ');
70321 202010
CO , 29190 ',' 202015 ');
29190 202015
CO , 70326 ',' 202010 ');
70326 202010
CO , 29258 ',' 202013 ');
29258 202013
MX , 29029 ',' 202013 ');
29029 202013
MX , 29027 ',' 202013 ');
29027 202013
MX , 29108 ',' 202015 ');
29108 202015
MX , 30478 ',' 202010 ');
30478 202010
MX , 30471 ',' 202010 ');
30471 202010
MX , 30476 ',' 202010 ');
30476 202010
MX , 29025 ',' 202013 ');
29025 202013
PE , 30365 ',' 202010 ');
30365 202010
PE , 29122 ',' 202010 ');
29122 202010
PE , 29350 ',' 202009 ');
29350 202009
PE , 29281 ',' 202009 ');
29281 202009
PE , 29361 ',' 202009 ');
29361 202009
PE , 29279 ',' 202009 ');
29279 202009
CL , 29222 ',' 202009 ');

In [184]:
final_data[final_data['flagSetHybris']]

,codpais,codsap,categories,images_url,isVariableSet,price,sale_price,stockLevelStatus,link_url,flag_ecommerce,name_prod,flagSetHybris,codsap_parent
0,CL,29030-202013,"{cyzone-40, cyzone}",https://cdn1-prd.beautymovers.com/sys-master/s...,False,23500,18790.0,inStock,https://cyzone.tiendabelcorp.com/cl/set-comple...,True,Set completo skin first,True,200097895
0,CL,30156-202010,"{cyzone-40, cyzone}",https://cdn1-prd.beautymovers.com/sys-master/s...,False,25200,20090.0,outOfStock,https://cyzone.tiendabelcorp.com/cl/set-sweet-...,True,Set sweet chic,True,200094149
0,CL,30152-202010,"{cyzone-40, cyzone}",https://cdn1-prd.beautymovers.com/sys-master/s...,False,25200,20090.0,outOfStock,https://cyzone.tiendabelcorp.com/cl/set-sweet-...,True,Set sweet black,True,200089168
0,CL,29010-202015,"{cyzone-40, cyzone}",https://cdn1-prd.beautymovers.com/sys-master/s...,False,28600,22790.0,inStock,https://cyzone.tiendabelcorp.com/cl/set-spray-...,True,Set spray fijador + paleta studio look,True,200096224
0,CL,30262-202010,"{cyzone-40, cyzone}",https://cdn1-prd.beautymovers.com/sys-master/s...,False,31400,24990.0,inStock,https://cyzone.tiendabelcorp.com/cl/set-studio...,True,Set studio look,True,200096224
...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,PE,29090-202009,"{esika-40, esika-08, esika}",https://cdn1-prd.beautymovers.com/sys-master/s...,False,280,223.9,inStock,https://esika.tiendabelcorp.com/pe/set-magnat-...,True,Set Magnat Opción 03,True,200100617
0,PE,29067-202009,"{esika-40, esika-08, esika}",https://cdn1-prd.beautymovers.com/sys-master/s...,False,150,119.9,inStock,https://esika.tiendabelcorp.com/pe/set-motion/...,True,Set Motion,True,200095635
0,PE,05695-202009,"{esika-40, esika-08, esika}",https://cdn1-prd.beautymovers.com/sys-master/s...,False,151,119.9,inStock,https://esika.tiendabelcorp.com/pe/set-you-tra...,True,Set You Travel,True,200095791
0,PE,29233-202010,"{esika-40, esika}",https://cdn1-prd.beautymovers.com/sys-master/s...,False,211,159.9,inStock,https://esika.tiendabelcorp.com/pe//p/29233-20...,True,None,True,200095850


In [186]:
final_data.to_csv('product_hybris_20201012_v3.csv', index=False)